# 0. Install and Import Dependencies

In [86]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import os
import csv
print(cv2.__version__)
import pandas as pd
print(pd.__version__)

4.10.0
2.2.2


# 1. Make Detections

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [90]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


# 3. Calculate Angles

In [92]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    #if angle >180.0:
        #angle = 360-angle
        
    return angle 

In [93]:


bowling_filename = "C:/Users/vmarw/Downloads/Bowling Videos-20241127T043536Z-001/Golden - Bowling videos/Novice bowler videos from Jan 2025 (nalin, AAyush)/Novice 4 and 5/test"
#code below is for video capture
#cap = cv2.VideoCapture (bowling_filename + ".mp4")

#code below is for Camera Logitech
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)   # or 1920
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)   # or 1080
cap.set(cv2.CAP_PROP_FPS, 30)

if not cap.isOpened():
    print("Error: Cannot open video")
    quit()


# Get the FPS using OpenCV's property
fps_opencv = cap.get(cv2.CAP_PROP_FPS)
print(f"FPS retrieved using OpenCV: {fps_opencv}")

# Get the total number of frames and video duration to calculate FPS manually
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
video_duration = total_frames / fps_opencv  # Using the initially retrieved FPS
fps_manual = total_frames / video_duration if video_duration > 0 else 0

print(f"Total frames: {total_frames}")
print(f"Video duration (seconds): {video_duration}")
print(f"Manually calculated FPS: {fps_manual:.2f}")

# Use the more accurate FPS value (if applicable)
fps = fps_manual if fps_manual > 0 else fps_opencv
print(f"Final FPS used: {fps}")

bowling_out_filename = bowling_filename + "_output.mp4"

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
out_frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
out_frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out_fps = cap.get(cv2.CAP_PROP_FPS)
print (out_fps)

frame_counter = 0
#Enter the Ball Release Frame
ball_release_frame = 76

frame_time = (frame_counter-ball_release_frame)/out_fps
# Create VideoWriter object

out_video = cv2.VideoWriter(bowling_out_filename, fourcc, out_fps, (out_frame_width, out_frame_height)) 

filename_angles = bowling_filename + "_" + str(ball_release_frame) + "_output.csv"
Header_row = ["frame_counter" , "frame_time", "left_arm_angle", "right_arm_angle", "left_leg_angle", "right_leg_angle", "left_foot_angle", "right_foot_angle", "left_wrist_angle", "right_wrist_angle", "Speed"]

with open(filename_angles, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(Header_row)

prev_right_wrist = None
prev_left_wrist = None
max_right_wrist_speed = 0
max_left_wrist_speed = 0

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            print("Error: Unable to capture a frame.")
            break
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            left_pinky = [landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].x,landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].y]
            right_pinky = [landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].y]

            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            left_foot = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
            right_foot = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
            
            # Calculate angle
            left_arm_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_arm_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

            left_leg_angle = calculate_angle(left_hip, left_knee, left_ankle)
            right_leg_angle = calculate_angle(right_hip, right_knee, right_ankle)

            left_wrist_angle = calculate_angle(left_elbow, left_wrist, left_pinky)
            right_wrist_angle = calculate_angle(right_elbow, right_wrist, right_pinky)

            left_foot_angle = calculate_angle(left_knee, left_ankle, left_foot)
            right_foot_angle = calculate_angle(right_knee, right_ankle, right_foot)

            left_arm_angle = "{:.2f}".format(left_arm_angle)
            right_arm_angle = "{:.2f}".format(right_arm_angle)
            left_leg_angle = "{:.2f}".format(left_leg_angle)
            right_leg_angle = "{:.2f}".format(right_leg_angle)

            left_foot_angle = "{:.2f}".format(left_foot_angle)
            right_foot_angle = "{:.2f}".format(right_foot_angle)

            left_wrist_angle = "{:.2f}".format(left_wrist_angle)
            right_wrist_angle = "{:.2f}".format(right_wrist_angle)

                       
            frame_time = (frame_counter-ball_release_frame)/out_fps
            frame_time = "{:.2f}".format(frame_time)
            #print (frame_time)

            # Get coordinates
            left_wrist = np.array([
                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y
            ])
            right_wrist = np.array([
                landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y
            ])

            # Calculate wrist speeds
            right_wrist_speed_mph = 0
            left_wrist_speed_mph = 0
            
            if prev_right_wrist is not None:
                # Convert pixel/frame to mph
                right_wrist_speed = np.linalg.norm(right_wrist - prev_right_wrist) * out_fps
                right_wrist_speed_mph = right_wrist_speed * 2.23694  # Convert m/s to mph
                if right_wrist_speed_mph < 0.1 or right_wrist_speed_mph > 28:  # Threshold to avoid noise
                    right_wrist_speed_mph = 0
            
            if prev_left_wrist is not None:
                left_wrist_speed = np.linalg.norm(left_wrist - prev_left_wrist) * out_fps
                left_wrist_speed_mph = left_wrist_speed * 2.23694  # Convert m/s to mph
                if left_wrist_speed_mph < 0.1 or left_wrist_speed_mph > 28:
                    left_wrist_speed_mph = 0

            prev_right_wrist = right_wrist
            prev_left_wrist = left_wrist
            # Round speeds to one decimal place
            right_wrist_speed_mph = round(right_wrist_speed_mph, 1)
            left_wrist_speed_mph = round(left_wrist_speed_mph, 1)
            
            # Check and update max speeds
            if right_wrist_speed_mph > max_right_wrist_speed:
                max_right_wrist_speed = right_wrist_speed_mph
            
            if left_wrist_speed_mph > max_left_wrist_speed:
                max_left_wrist_speed = left_wrist_speed_mph
            
            print(frame_time, right_wrist_speed_mph, max_right_wrist_speed)
                        
            out_row = [frame_counter, frame_time, left_arm_angle , right_arm_angle, left_leg_angle, right_leg_angle, left_foot_angle, right_foot_angle, left_wrist_angle, right_wrist_angle, right_wrist_speed_mph ]
            
            with open(filename_angles, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(out_row)
            
            # Visualize angle
            
            cv2.putText(image, str("LEFT_ARM " + left_arm_angle), 
                           (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            cv2.putText(image, str("RIGHT_ARM " + right_arm_angle), 
                           (10, 50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )

            cv2.putText(image, str("LEFT_WRIST " + left_wrist_angle), 
                           (10, 70), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str("RIGHT_WRIST " + right_wrist_angle), 
                           (10, 90), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            cv2.putText(image, str("LEFT_LEG " +left_leg_angle), 
                           (10, 110), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str("RIGHT_LEG " +right_leg_angle), 
                           (10, 130), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            cv2.putText(image, str("FRAME_COUNTER " + str(frame_counter)), 
                           (10, 150), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            cv2.putText(image, str("WRIST_SPEED mph " + str(max_right_wrist_speed)), 
                           (10, 170), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
              
        cv2.imshow('Mediapipe Feed', image)
        frame_counter += 1
        out_video.write(image)
        

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    df = pd.read_csv(filename_angles)
    print(df.head())
    out_video.release()
    cap.release()
    cv2.destroyAllWindows()

FPS retrieved using OpenCV: 30.00003000003
Total frames: -1
Video duration (seconds): -0.033333299999999996
Manually calculated FPS: 0.00
Final FPS used: 30.00003000003
30.00003000003
-2.23 0 0
-2.20 0.3 0.3
-2.17 0.8 0.8
-2.13 2.4 2.4
-2.10 0.9 2.4
-2.07 3.6 3.6
-1.93 26.0 26.0
-1.90 0.8 26.0
-1.87 0.8 26.0
-1.83 0.2 26.0
-1.80 0.2 26.0
-1.77 0.4 26.0
-1.73 0.4 26.0
-1.70 0.8 26.0
-1.67 3.2 26.0
-1.63 4.2 26.0
-1.60 1.2 26.0
-1.57 1.8 26.0
-1.53 4.0 26.0
-1.50 1.5 26.0
-1.47 0.4 26.0
-1.43 0 26.0
-1.40 0.5 26.0
-1.37 0.5 26.0
-1.33 0.6 26.0
-1.30 14.4 26.0
-1.27 10.8 26.0
-1.23 3.1 26.0
-1.20 22.6 26.0
-1.17 18.6 26.0
-1.13 17.7 26.0
-1.10 13.9 26.0
-1.07 8.4 26.0
-1.03 0.7 26.0
-1.00 1.2 26.0
-0.97 12.9 26.0
-0.93 4.1 26.0
-0.90 0 26.0
-0.87 3.8 26.0
-0.83 10.7 26.0
-0.80 5.0 26.0
-0.77 4.3 26.0
-0.73 8.8 26.0
-0.70 4.0 26.0
-0.67 7.7 26.0
-0.63 21.0 26.0
-0.60 6.9 26.0
-0.57 1.4 26.0
-0.53 0.8 26.0
-0.50 2.5 26.0
-0.47 0.1 26.0
-0.43 1.3 26.0
-0.40 1.6 26.0
-0.37 1.3 26.0
-0.33 4.0 